In [4]:
import sys
sys.path.append('../chromdyn')
from Topology import TopologyGenerator
from ChromatinDynamics import ChromatinDynamics

In [2]:
#Initialize generator object
generator = TopologyGenerator()

#generate topology
generator.gen_top([100])


In [6]:
#Initialize simulation object
sim = ChromatinDynamics(generator.topology, name = 'test', platform_name = "OpenCL", output_dir = "output", console_stream = True)

2025-08-06 14:52:57,904 | INFO | ChromatinDynamics | ************************************************************
2025-08-06 14:52:57,905 | INFO | ChromatinDynamics |                      Chromatin Dynamics                     
2025-08-06 14:52:57,906 | INFO | ChromatinDynamics | ************************************************************
2025-08-06 14:52:57,906 | INFO | ChromatinDynamics | System initialized with 100 particles. Output directory: output
2025-08-06 14:52:57,907 | INFO | Platforms | Platform 'OpenCL' is available and selected.
2025-08-06 14:52:57,907 | INFO | ChromatinDynamics | force_field_manager initialized. Use this to add forces before running setup.


In [7]:
#Add forces
sim.force_field_manager.add_harmonic_bonds()
sim.force_field_manager.add_self_avoidance()
sim.force_field_manager.add_flat_bottom_harmonic()

2025-08-06 14:54:23,893 | INFO | Forcefield | Adding 99 harmonic bonds with r0=1.0, k=10.0, group=0
2025-08-06 14:54:23,893 | INFO | Forcefield | HarmonicBonds force successfully added to system.
2025-08-06 14:54:23,894 | INFO | Forcefield | --------------------------------------------------
2025-08-06 14:54:23,895 | INFO | Forcefield | Adding Self-avoidance force with parameters:
2025-08-06 14:54:23,896 | INFO | Forcefield | Ecut=4.0, k_rep=5.0, r_rep=1.0, cutoff=5.0, group=2
2025-08-06 14:54:23,896 | INFO | Forcefield | Added exclusions from bonded monomers.
2025-08-06 14:54:23,897 | INFO | Forcefield | SelfAvoidance force successfully added to system.
2025-08-06 14:54:23,897 | INFO | Forcefield | --------------------------------------------------
2025-08-06 14:54:23,897 | INFO | Forcefield | --------------------------------------------------
2025-08-06 14:54:23,898 | INFO | Forcefield | Adding Flat-Bottom Harmonic potential with parameters:
2025-08-06 14:54:23,898 | INFO | Forcefiel

In [ ]:
#set up the simulation
sim.simulation_setup(
            init_struct='randomwalk',
            integrator='langevin',
            temperature=120.0,
            timestep=0.01,
            save_pos=True,
            save_energy=True,
            energy_report_interval=3_000,
            pos_report_interval=1000,              
            )

2025-08-06 14:56:19,812 | INFO | Integrators | Creating integrator ...
2025-08-06 14:56:19,813 | INFO | Integrators | LangevinIntegrator: temperatute=120.0 | friction=0.1 | timestep=0.01
2025-08-06 14:56:19,994 | INFO | ChromatinDynamics | Setting up simulation context...
2025-08-06 14:56:19,994 | INFO | Utilities | Random walk created. Position shape: (100, 3)
2025-08-06 14:56:19,997 | INFO | ChromatinDynamics | Simulation context initialized.
2025-08-06 14:56:19,997 | INFO | ChromatinDynamics | ------------------------------------------------------------------------------------------------------------------------
2025-08-06 14:56:19,997 | INFO | ChromatinDynamics | Index  Force Class                    Force Name           Group    Particles    Bonds        Exclusions   P.E./Particle       
2025-08-06 14:56:19,998 | INFO | ChromatinDynamics | ------------------------------------------------------------------------------------------------------------------------
2025-08-06 14:56:20,00

In [9]:
#collapse run
sim.run(10_000, report=False)

2025-08-06 14:57:34,722 | INFO | ChromatinDynamics | ------------------------------------------------------------
2025-08-06 14:57:34,723 | INFO | ChromatinDynamics | Running simulation for 10000 steps...
2025-08-06 14:57:34,723 | INFO | ChromatinDynamics | Paused reporter: position
2025-08-06 14:57:34,724 | INFO | ChromatinDynamics | Paused reporter: energy
2025-08-06 14:57:35,660 | INFO | ChromatinDynamics | Completed 10000 steps in 0.94s (10694 steps/s)
2025-08-06 14:57:35,660 | INFO | ChromatinDynamics | ------------------------------------------------------------
2025-08-06 14:57:35,661 | INFO | ChromatinDynamics | Resumed reporter: position
2025-08-06 14:57:35,661 | INFO | ChromatinDynamics | Resumed reporter: energy


10693.987987090635

In [10]:
# save structures while running
sim.run(10_000, report=True)

2025-08-06 14:58:15,844 | INFO | ChromatinDynamics | ------------------------------------------------------------
2025-08-06 14:58:15,844 | INFO | ChromatinDynamics | Running simulation for 10000 steps...
2025-08-06 14:58:16,694 | INFO | ChromatinDynamics | Completed 10000 steps in 0.85s (11781 steps/s)
2025-08-06 14:58:16,694 | INFO | ChromatinDynamics | ------------------------------------------------------------


11780.521810642258